# Conditioning, stability, and accuracy

Understanding why some numerical calculations go awry. 



## A problematic problem

Problem 5 from the homework asks for the root of $f(x) = x^3 - 2x^2 + 4x/3 - 8/27$. Let's investigate this problem with the secant method (repeated here from the previous lecture). 

In [1]:
function secantsearch(f, a::AbstractFloat, b::AbstractFloat, diagnostics=false) 
# return solution x of f(x) == 0, with a,b serving as initial guesses
    
    fa = f(a)           # cache values of f(a), f(b)     
    fb = f(b)
    
    # set c to better of a,b (and fc, too)
    c,fc = abs(fa) < abs(fb) ? (a,fa) : (b,fb)
    
    floattype = typeof((a+b)/2) # this'll eval to bigger of a,b types
    ϵ = eps(floattype)  # machine epsilon
    n = 1               # iteration count
    N = 1024            # max iterations
    
    # start vector for storing root iterates xₙ, assign x₁ = c
    x = zeros(floattype, 1)
    x[1] = c
    
    # stop when f(c) is small, |b-a| is small relative to a and b
    # or when we reach maximium number of iterations
    while abs(fc) > 10ϵ && abs(b-a)/(abs(a)+abs(b)) > 10ϵ && n < N
    
        # compute new values for c, fc by secant approximation
        c = a - fa*(b-a)/(fb-fa)
        fc = f(c)
        
        # print diagnostics if they're turned on
        diagnostics && println("n = $n, c = $c, f(c) = $(f(c))")
        
        # error and exit if f(c) is bigger than both f(a) and f(b)
        if abs(fc) > abs(fa) && abs(fc) > abs(fb)
            println("secantsearch error: f(c) is bigger than f(a), f(b)")
            break # break out of while loop, don't store bad value c
        end
        
        # replace whichever of a, b has bigger f(a), f(b)
        if abs(fa) > abs(fb)
            a,fa = c,fc
        else
            b,fb = c,fc
        end      
        
        # store c in x, then increment iteration counter
        push!(x,c)   
        n += 1
    end
    
    # return c and vector x
    c, x
    
end

secantsearch (generic function with 2 methods)

In [2]:
f(x) = x.^3 - 2x.^2 + 4/3*x - 8/27

f (generic function with 1 method)

In [6]:
f(0), f(2)

(-0.2962962962962963, 2.3703703703703702)

In [7]:
r,x = secantsearch(f, 0.0, 1.0,true)

n = 1, c = 0.8888888888888891, f(c) = 0.010973936899862813
n = 2, c = 0.8421052631578949, f(c) = 0.005399772129616065
n = 3, c = 0.796785304247992, f(c) = 0.002203020416244428
n = 4, c = 0.7655533526628087, f(c) = 0.0009669710403885645
n = 5, c = 0.7411203540240782, f(c) = 0.0004127229640434882
n = 6, c = 0.7229262275146178, f(c) = 0.000178069284412663
n = 7, c = 0.7091194332075769, f(c) = 7.650996304064961e-5
n = 8, c = 0.6987180509624937, f(c) = 3.292610616567604e-5
n = 9, c = 0.6908601641064995, f(c) = 1.4161066591356075e-5
n = 10, c = 0.6849301974472575, f(c) = 6.091920437989451e-6
n = 11, c = 0.6804532819965489, f(c) = 2.6204324846679228e-6
n = 12, c = 0.6770739084149917, f(c) = 1.1272154389896372e-6
n = 13, c = 0.6745228512285835, f(c) = 4.848808504709368e-7
n = 14, c = 0.6725971277812471, f(c) = 2.085765059556266e-7
n = 15, c = 0.6711434384851674, f(c) = 8.972115939087644e-8
n = 16, c = 0.6700460819689645, f(c) = 3.859443598042134e-8
n = 17, c = 0.6692177117354121, f(c) = 1.6601

(0.6666787536187389, [1.0, 0.888889, 0.842105, 0.796785, 0.765553, 0.74112, 0.722926, 0.709119, 0.698718, 0.69086  …  0.66682, 0.666782, 0.666754, 0.666733, 0.666716, 0.666704, 0.666695, 0.666688, 0.666683, 0.666679])

Hmmm, that's interesting. The secant method took 36 iterations before stopping on this $f(x)$, when it only took 7 iterations for $f(x) = x^2 - 4$. Also, `c` is still changing in the 5th digit between the last two iterations! It looks like we only have 5 or 6 digits of accuracy in our solution $r \approx 0.66667$. That decimal expansion is pretty suggestive, though. Let's check $f(2/3)$

In [8]:
f(2/3)

0.0